In [2]:
import numpy as np
import pandas as pd

In [3]:
import sportsdataverse

In [4]:
mbb_df = sportsdataverse.mbb.load_mbb_pbp(seasons=[2019], return_as_pandas=True)

100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.58s/it]


In [5]:
mbb_df.head(5)

,id,sequence_number,type_id,type_text,text,away_score,home_score,period_number,period_display_value,clock_display_value,...,period,athlete_id_1,lag_qtr,lag_game_half,athlete_id_2,media_id,coordinate_x,coordinate_y,game_date,game_date_time
0,4.011234e+17,101799901,615,Jumpball,Jump Ball won by Texas Tech,0,0,1,1st Half,20:00,...,1,<NA>,<NA>,<NA>,<NA>,<NA>,-41.75,0.0,2019-04-08,2019-04-08 21:20:00-04:00
1,4.011234e+17,101806901,20558,JumpShot,Jarrett Culver missed Jumper.,0,0,1,1st Half,19:30,...,1,4277928,1,1,<NA>,<NA>,-41.75,19.0,2019-04-08,2019-04-08 21:20:00-04:00
2,4.011234e+17,101806902,587,Defensive Rebound,Ty Jerome Defensive Rebound.,0,0,1,1st Half,19:30,...,1,4065733,1,1,<NA>,<NA>,41.75,-0.0,2019-04-08,2019-04-08 21:20:00-04:00
3,4.011234e+17,101809901,20572,LayUpShot,De'Andre Hunter missed Layup.,0,0,1,1st Half,19:00,...,1,4065732,1,1,<NA>,<NA>,40.75,-2.0,2019-04-08,2019-04-08 21:20:00-04:00
4,4.011234e+17,101809902,587,Defensive Rebound,Tariq Owens Defensive Rebound.,0,0,1,1st Half,19:00,...,1,3149059,1,1,<NA>,<NA>,-41.75,0.0,2019-04-08,2019-04-08 21:20:00-04:00


In [6]:
mbb_df.columns

Index(['id', 'sequence_number', 'type_id', 'type_text', 'text', 'away_score',
       'home_score', 'period_number', 'period_display_value',
       'clock_display_value', 'scoring_play', 'score_value', 'team_id',
       'shooting_play', 'coordinate_x_raw', 'coordinate_y_raw', 'season',
       'season_type', 'away_team_id', 'away_team_name', 'away_team_mascot',
       'away_team_abbrev', 'away_team_name_alt', 'home_team_id',
       'home_team_name', 'home_team_mascot', 'home_team_abbrev',
       'home_team_name_alt', 'home_team_spread', 'game_spread',
       'home_favorite', 'game_spread_available', 'game_id', 'qtr', 'time',
       'clock_minutes', 'clock_seconds', 'half', 'game_half', 'lead_qtr',
       'lead_game_half', 'start_quarter_seconds_remaining',
       'start_half_seconds_remaining', 'start_game_seconds_remaining',
       'game_play_number', 'end_quarter_seconds_remaining',
       'end_half_seconds_remaining', 'end_game_seconds_remaining', 'period',
       'athlete_id_1', 'lag

In [7]:
# GET ONLY NEEDED COLUMNS
mbb_df = mbb_df[['id', 'away_score',
       'home_score','start_quarter_seconds_remaining','period', 'away_team_id', 'team_id', 'game_id' ]]

In [8]:
# Get Score Difference Column
mbb_copy = mbb_df.copy()
mbb_copy['score_difference'] = mbb_copy['home_score'] - mbb_copy['away_score'] # gets score_diff column
mbb = mbb_copy

In [9]:
# Get total seconds in game
mbb['seconds_remaining'] = mbb.apply(
    lambda row: row['start_quarter_seconds_remaining'] if row['period'] != 1 else row['start_quarter_seconds_remaining'] + 1200,
    axis=1
)

In [10]:
# Get possession column (Generates warning, still works)
mbb = mbb.dropna() # DROP NA DROPS TOO MANY COLUMNS
mbb['possession'] = mbb.apply(
    lambda row: 1 if row['away_team_id'] != row['team_id'] else 0,
    axis=1
)

/var/folders/p1/4fyms5s95238j0rrzgn67dr00000gn/T/ipykernel_25255/540938573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbb['possession'] = mbb.apply(


In [11]:
# Home_win_Column (Generates warning but still works)
groups = mbb.groupby('game_id')
for group_name, group_data in groups:
    if group_data.iloc[-1]['score_difference'] > 0:
        mbb.loc[mbb['game_id'] == group_name, 'home_win'] = 1
    else:
        mbb.loc[mbb['game_id'] == group_name, 'home_win'] = 0

/var/folders/p1/4fyms5s95238j0rrzgn67dr00000gn/T/ipykernel_25255/520789322.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbb.loc[mbb['game_id'] == group_name, 'home_win'] = 1


In [12]:
mbb # only need score difference, 

,id,away_score,home_score,start_quarter_seconds_remaining,period,away_team_id,team_id,game_id,score_difference,seconds_remaining,possession,home_win
0,4.011234e+17,0,0,1200,1,2641,2641,401123374,0,2400.0,0,1.0
1,4.011234e+17,0,0,1170,1,2641,2641,401123374,0,2370.0,0,1.0
2,4.011234e+17,0,0,1170,1,2641,258,401123374,0,2370.0,1,1.0
3,4.011234e+17,0,0,1140,1,2641,258,401123374,0,2340.0,1,1.0
4,4.011234e+17,0,0,1140,1,2641,2641,401123374,0,2340.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1787326,4.010837e+17,74,42,52,2,2539,302,401083745,-32,52.0,1,0.0
1787327,4.010837e+17,74,42,41,2,2539,302,401083745,-32,41.0,1,0.0
1787328,4.010837e+17,76,42,32,2,2539,2539,401083745,-34,32.0,0,0.0
1787329,4.010837e+17,76,42,11,2,2539,302,401083745,-34,11.0,1,0.0


In [13]:
mbb_condensed = mbb[['away_score', 'home_score', 'score_difference', 'seconds_remaining', 'possession', 'home_win']]

In [14]:
mbb_condensed # final dataframe?
# include actual score
# team fouls?

,away_score,home_score,score_difference,seconds_remaining,possession,home_win
0,0,0,0,2400.0,0,1.0
1,0,0,0,2370.0,0,1.0
2,0,0,0,2370.0,1,1.0
3,0,0,0,2340.0,1,1.0
4,0,0,0,2340.0,0,1.0
...,...,...,...,...,...,...
1787326,74,42,-32,52.0,1,0.0
1787327,74,42,-32,41.0,1,0.0
1787328,76,42,-34,32.0,0,0.0
1787329,76,42,-34,11.0,1,0.0


In [ ]:
# https://stackoverflow.com/questions/41511945/python-pandas-linear-regression-groupby
# Regression groupby